# Load packages

In [1]:
# ! pip install supervision
# ! pip install -q supervision==0.6.0
# ! pip install contextily
# ! pip install scikit-image
# ! pip install pykrige
# ! pip install selenium

In [1]:
import sys
print(sys.executable)


e:\ProgramData\Anaconda3\envs\street_mapping_env\python.exe


In [2]:
# import matplotlib.patches as patches
import supervision as sv

import sys
import numpy as np
import glob
import os
import bz2
import pandas as pd
import geopandas as gpd
import pickle
from PIL import Image
import street_triangulation as tri
import supervision as sv
import cv2
import math
import pdb
import contextily as ctx
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import matplotlib.pyplot as plt



pd.set_option('display.max_columns', None)

# sys.path.append(r'E:\Research\StreetView\gsv_pano')
sys.path.append(r'D:\Research\StreetView\gsv_pano')

from pano import GSV_pano
import utils

%load_ext autoreload
%autoreload 2

# Set directories

In [4]:
detect_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_segmented'
depthmap_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_depth'
image_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail'
pano_meta_fname = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail.csv"
annotated_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_segmented'
local_crs = 6569 # EPSG:6569, NAD83(2011) / South Carolina
save_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_trunks'

# detect_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_segmented'
# depthmap_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_depth'
# image_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail'
# pano_meta_fname = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail.csv"
# annotated_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_segmented'
# local_crs = 6569 # EPSG:6569, NAD83(2011) / South Carolina
# save_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_trunks'


In [27]:
img_height = 768
img_width = 1024

camera_dis_thres =  30  #20
trunk_near_thres = 3

# state colleage
# detect_dir = r'G:\My Drive\Research\street_mapping\thumbnals_state_college'
# depthmap_dir = r'G:\My Drive\Research\street_mapping\thumbnals_state_college\depthmaps'
# image_dir = r'G:\My Drive\Research\street_mapping\thumbnals_state_college'
# pano_meta_fname = r"G:\My Drive\Research\street_mapping\state_college_json\jsons.csv"
# annotated_dir = r'G:\My Drive\Research\street_mapping\thumbnals_state_college'
# local_crs = 6562 # EPSG:6562, NAD83(2011) / Pennsylvania North

# save_dir = r'G:\My Drive\Research\street_mapping\state_college_trunks'
# os.makedirs(save_dir, exist_ok=True)
# CLASSES = ['vehicle', 'tree_trunk', 'person', 'sidewalk', 'road', 'building', 'grass']   # , 'tree root collar'
# trunk_index = 1


# Columbia
# detect_dir = r'G:\My Drive\Research\street_mapping\South_Harden_St\South_Harden_St_segmented'
# depthmap_dir = r'G:\My Drive\Research\street_mapping\South_Harden_St\thumbnails\depthmaps'
# image_dir = r'D:\Research\street_image_mapping\South_Harden_St\thumbnails'
# pano_meta_fname = r"G:\My Drive\Research\street_mapping\South_Harden_St\thumbnails.csv"
# annotated_dir = r'G:\My Drive\Research\street_mapping\South_Harden_St\South_Harden_St_segmented'
# local_crs = 6569 # EPSG:6569, NAD83(2011) / South Carolina
# save_dir = r'G:\My Drive\Research\street_mapping\South_Harden_St\trunks'

# detect_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_segmented'
# depthmap_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_depth'
# image_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail'
# pano_meta_fname = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail.csv"
# annotated_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_segmented'
# local_crs = 6569 # EPSG:6569, NAD83(2011) / South Carolina
# save_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_trunks'



os.makedirs(save_dir, exist_ok=True)


CLASSES = ['tree_trunk']   # , 'tree root collar'
trunk_index = 0

In [6]:
pano_meta_df = pd.read_csv(pano_meta_fname).set_index('panoId')
pano_meta_df

,image_width,image_height,tile_width,tile_height,image_date,imagery_type,projection_type,pano_yaw_deg,tilt_yaw_deg,tilt_pitch_deg,zoomLevels,lat,lng,original_lat,original_lng,elevation_wgs84_m,description,streetRange,region,country,elevation_egm96_m,links_cnt,link_forward_yawDeg,link_f_panoId,link_f_road_argb,link_f_description,link_backward_yawDeg,link_b_panoId,link_b_road_argb,link_b_description,link_others
panoId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
znuYMYuRAnWodO_nKEcpAw,13312,6656,512,512,2014-8,1,spherical,5.234205,90.486496,1.345668,NaN,33.990883,-81.006261,NaN,NaN,NaN,101 S Woodrow St,NaN,Columbia; South Carolina,US,89.712708,4,4.480649,iwIR7ReTsovIl2NEO1n1jA,NaN,Woodrow St,279.834473,oUNvMa7PVtjhDnx-1NnsTQ,NaN,Heyward St,89.1512451171875|_dSD7M9ggy25Y8ocd3bOVw||Heywa...
bD0H1wJkNsf920LkpU9bXA,16384,8192,512,512,2022-2,1,spherical,183.540970,88.687180,0.508491,NaN,33.990815,-81.006311,NaN,NaN,NaN,101 S Woodrow St,NaN,Columbia; South Carolina,US,89.003746,2,184.599854,p4RFDXvkNmC42i42QrU10Q,NaN,S Woodrow St,4.619255,znuYMYuRAnWodO_nKEcpAw,NaN,S Woodrow St,NaN
p4RFDXvkNmC42i42QrU10Q,16384,8192,512,512,2022-2,1,spherical,183.230072,89.076469,0.397373,NaN,33.990722,-81.006318,NaN,NaN,NaN,100 S Woodrow St,NaN,Columbia; South Carolina,US,88.850060,2,184.667160,HLEanEQavzLGOgt1CAy8tA,NaN,S Woodrow St,4.599847,bD0H1wJkNsf920LkpU9bXA,NaN,S Woodrow St,NaN
HLEanEQavzLGOgt1CAy8tA,16384,8192,512,512,2021-12,1,spherical,184.593185,89.518768,1.281742,NaN,33.990650,-81.006327,NaN,NaN,NaN,100 S Woodrow St,NaN,Columbia; South Carolina,US,88.781258,2,184.638870,ye0Bj3kdK30YplYkpneKdQ,NaN,S Woodrow St,4.667163,p4RFDXvkNmC42i42QrU10Q,NaN,S Woodrow St,NaN
ye0Bj3kdK30YplYkpneKdQ,16384,8192,512,512,2022-2,1,spherical,183.486938,88.766350,0.703531,NaN,33.990631,-81.006325,NaN,NaN,NaN,100 S Woodrow St,NaN,Columbia; South Carolina,US,88.756538,2,184.591965,o3bXwz6__nb8XqfLceHOxA,NaN,S Woodrow St,4.638868,HLEanEQavzLGOgt1CAy8tA,NaN,S Woodrow St,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64nVR_sX79QOPrzaB4geGA,16384,8192,512,512,2022-2,1,spherical,186.775589,89.057243,0.207718,NaN,33.991285,-81.010097,NaN,NaN,NaN,101 Queen St,NaN,Columbia; South Carolina,US,89.012497,2,184.367188,3KcssIfHMS2dkud-EWFGvw,NaN,Queen St,4.360662,wccJ8OQ4GUVc_wiXBd8FgA,NaN,Queen St,NaN
wccJ8OQ4GUVc_wiXBd8FgA,16384,8192,512,512,2022-2,1,spherical,189.261673,89.076744,359.052429,NaN,33.991405,-81.010068,NaN,NaN,NaN,103 Queen St,NaN,Columbia; South Carolina,US,89.108795,2,184.360657,64nVR_sX79QOPrzaB4geGA,NaN,Queen St,4.336567,skKFC4dwbevwx7Jid_sn9g,NaN,Queen St,NaN
iwIR7ReTsovIl2NEO1n1jA,16384,8192,512,512,2022-2,1,spherical,184.167801,89.896645,359.674896,NaN,33.990953,-81.006299,NaN,NaN,NaN,101 Woodrow St,NaN,Columbia; South Carolina,US,89.126038,2,184.480652,znuYMYuRAnWodO_nKEcpAw,NaN,S Woodrow St,4.475543,kC_WDnM37rYXWVuZEQoTWQ,NaN,Woodrow St,NaN


In [7]:
detect_files = glob.glob(os.path.join(detect_dir, "*.bz2"))
detect_files = sorted(detect_files)
print("Found file count:", len(detect_files))
# ['D:\\OneDrive_PSU\\OneDrive - The Pennsylvania State University\\Research_doc\\street_image_mapping\\Heyward_st_pano_thumnail_segmented\\piIyY_Bgq2dmllHQlN7FRw_0.0_0.0_0_226.73_R135.pkl.bz2']
# detect_files = ['D:\\OneDrive_PSU\\OneDrive - The Pennsylvania State University\\Research_doc\\street_image_mapping\\Heyward_st_pano_thumnail_segmented\\piIyY_Bgq2dmllHQlN7FRw_0.0_0.0_0_226.73_R135.pkl.bz2', 'D:\\OneDrive_PSU\\OneDrive - The Pennsylvania State University\\Research_doc\\street_image_mapping\\Heyward_st_pano_thumnail_segmented\\XShHFvEc3Mi2fhPmjY7OGw_0.0_0.0_0_180.97_R90.pkl.bz2'] + detect_files  # add a file on the top for testing
# detect_files
detect_files = ['D:\\OneDrive_PSU\\OneDrive - The Pennsylvania State University\\Research_doc\\street_image_mapping\\Heyward_st_pano_thumnail_segmented\\L9Jp_XEhrMmgm35YhruiRQ_0.0_0.0_0_273.58_R90.pkl.bz2'] + detect_files  # add a file on the top for testing


print("Found file count:", len(detect_files))


Found file count: 762
Found file count: 763


In [19]:
# detect_files[458:]  # detect_files[458] = XShHFvEc3Mi2fhPmjY7OGw_0.0_0.0_0_180.97_R90.pkl.bz2

# Compute measurement location

In [51]:
pair_trunk_df_list = []

for idx, detect_file in enumerate(detect_files[:]): # 458:460
    try:
        # print("detect_file: ", detect_file)
        # detect_file = os.path.join(detect_dir, r'l-PhEd7ACdyR12n08p1Q0Q_0.0_0.0_0_33.59_R90.pkl.bz2')  # [68:69]
        # detect_file = os.path.join(detect_dir, r'XciMXMjYPg0jhKh8cKG1eA_0.0_0.0_0_78.39_R135.pkl.bz2')
        # detect_file = os.path.join(detect_dir, r'vFu4cCea3kkheGt941HrBA_0.0_0.0_0_48.96_L45.pkl.bz2')
        
        gdf_list = []

        basename = os.path.basename(detect_file).replace('.pkl.bz2', '.jpg')
        direction_str = basename.replace(".pkl.bz2", "").split("_")[-1]

        image_fname = os.path.join(image_dir, basename)

        
        # Step 1: form image pairs. 1) select a non-look behind direction. 2) 
        # non-look behind direction: R90, R45, L90, L45.
        # look behind direction:     L135, R135. 

        # skip the look behind image
        if direction_str in ['L135', 'R135']:  #  I make a mistake when naming the thumbnails in State College: the "L135" and "L45" should be switched.
            print("skip: ", direction_str, basename)
 
        print(f"Processing: {idx + 1} / {len(detect_files)}: {basename}" )


        # find image pairs
        panoID = basename[:22]
        pano_meta = pano_meta_df.loc[panoID]
        # print("pano_meta:", pano_meta)

        # obtain image pairs: R90 <-> R135, R45 <-> R90, L90 <-> L135, L45 <-> L90, 
        image_pairs = tri.form_image_pairs(detect_file, pano_meta, detect_dir=None)
        # print("idx:", idx)


        if len(image_pairs) == 0:
            print("    No image_pairs, skip. \n")
            continue

        else:
            for img1, img2 in image_pairs:
                print(f"    image pairs: {os.path.basename(img1), os.path.basename(img2)}")

        print()
        # print("idx:", idx)

        # extract the trunks in the current image
        trunk_df = tri.get_trunks_in_an_image(detect_file, trunk_index, image_dir, pano_meta_df, depthmap_dir, local_crs)
        # print("trunk_df:", trunk_df)
        # print("idx:", idx, trunk_df)  # passed

        # Step 2:  form trunk pairs
        # extract the trunks in the paire image
        for pair in image_pairs:
            print("pair[1]:", pair[1])
            paired_trunk_df = tri.get_trunks_in_an_image(pair[1], trunk_index, image_dir, pano_meta_df, depthmap_dir, local_crs)
            # print("paired_trunk_df:", paired_trunk_df)  

            # merge trunks into a GeoDataFrame
            all_gdf = tri.merge_trunk_df_pair(trunk_df, paired_trunk_df, local_crs)
            # print("all_gdf:", all_gdf)
    
            all_trunk_gdf = all_gdf.query("Type == 'trunk' ")
            # all_trunk_gdf['depth_x'] = all_trunk_gdf['depth_x'].astype('float')
            # all_trunk_gdf['depth_y'] = all_trunk_gdf['depth_y'].astype('float')
            all_trunk_gdf = all_trunk_gdf.reset_index()
    
            # all_camera_gdf = all_gdf.query("Type == 'camera' ")
            # print("all_camera_gdf:", all_camera_gdf)
    
            # form trunck pairs
            trunk_pairs_df = tri.form_trunk_pairs(all_trunk_gdf, near_thres=trunk_near_thres, camera_dis_thres=camera_dis_thres)
    
            if trunk_pairs_df is None:
                print("    No trunk pairs.")
                continue
    
            # print("trunk_pairs_df:", trunk_pairs_df)
    
            pair_cnt = int(len(trunk_pairs_df) / 2)
    
            # print("all_trunk_gdf:", all_trunk_gdf)
    
            # draw the trunks
            for p in range(pair_cnt):
                a_pair_trunk_df = trunk_pairs_df.iloc[2 * p:2 * p + 2].copy()
                a_pair_trunk_df = tri.triangulate_trunk_location(a_pair_trunk_df)  # compute location
                a_pair_trunk_df = tri.compute_diameter(a_pair_trunk_df)  # compute diameters
    
                pair_trunk_df_list.append(a_pair_trunk_df)
    
                tri.show_detailed_images(a_pair_trunk_df, local_crs, image_dir, annotated_dir, save_dir)
    
            print()
            # break

    except Exception as e:
        print()
        print("Error:", e, idx, basename)
        print()
        # break

all_pair_trunk_df = pd.concat(pair_trunk_df_list)
csv_fname = os.path.join(save_dir, "detected_trunks.csv")
all_pair_trunk_df.to_csv(csv_fname, index=False)

geometry = gpd.points_from_xy(all_pair_trunk_df['tri_x'], all_pair_trunk_df['tri_y'])
localized_trunk_gdf = gpd.GeoDataFrame(data=all_pair_trunk_df, geometry=geometry).set_crs(local_crs)
localized_trunk_fname = os.path.join(save_dir, "localized_trunks.shp")
localized_trunk_gdf.to_file(localized_trunk_fname)

Processing: 1 / 763: L9Jp_XEhrMmgm35YhruiRQ_0.0_0.0_0_273.58_R90.jpg
    image pairs: ('L9Jp_XEhrMmgm35YhruiRQ_0.0_0.0_0_273.58_R90.pkl.bz2', 'HkX4nzLQ1XWyjEprr4nFXA_0.0_0.0_0_321.20_R135.pkl.bz2')

pair[1]: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_segmented\HkX4nzLQ1XWyjEprr4nFXA_0.0_0.0_0_321.20_R135.pkl.bz2


C:\Users\USER\.conda\envs\geo\lib\site-packages\contextily\tile.py:614: UserWarning: The inferred zoom level of 20 is not valid for the current tile provider (valid zooms: 0 - 19).
  warnings.warn(msg)



Processing: 2 / 763: -apTLrvpSNEjvG8b5eWZ2A_0.0_0.0_0_138.68_R45.jpg
    image pairs: ('-apTLrvpSNEjvG8b5eWZ2A_0.0_0.0_0_138.68_R45.pkl.bz2', 'k5-O4GqXpRuptfifbbEKiA_0.0_0.0_0_228.49_R135.pkl.bz2')
    image pairs: ('-apTLrvpSNEjvG8b5eWZ2A_0.0_0.0_0_138.68_R45.pkl.bz2', 'k5-O4GqXpRuptfifbbEKiA_0.0_0.0_0_183.49_R90.pkl.bz2')

pair[1]: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_segmented\k5-O4GqXpRuptfifbbEKiA_0.0_0.0_0_228.49_R135.pkl.bz2
    No trunk pairs.
pair[1]: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_segmented\k5-O4GqXpRuptfifbbEKiA_0.0_0.0_0_183.49_R90.pkl.bz2


C:\Users\USER\.conda\envs\geo\lib\site-packages\contextily\tile.py:614: UserWarning: The inferred zoom level of 20 is not valid for the current tile provider (valid zooms: 0 - 19).
  warnings.warn(msg)



Processing: 3 / 763: -apTLrvpSNEjvG8b5eWZ2A_0.0_0.0_0_183.68_R90.jpg
    image pairs: ('-apTLrvpSNEjvG8b5eWZ2A_0.0_0.0_0_183.68_R90.pkl.bz2', 'k5-O4GqXpRuptfifbbEKiA_0.0_0.0_0_228.49_R135.pkl.bz2')

pair[1]: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_segmented\k5-O4GqXpRuptfifbbEKiA_0.0_0.0_0_228.49_R135.pkl.bz2


...

C:\Users\USER\.conda\envs\geo\lib\site-packages\contextily\tile.py:614: UserWarning: The inferred zoom level of 20 is not valid for the current tile provider (valid zooms: 0 - 19).
  warnings.warn(msg)



pair[1]: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_segmented\c_EHRnqMlsR0KzHFaBKF2g_0.0_0.0_0_3.83_L90.pkl.bz2


C:\Users\USER\.conda\envs\geo\lib\site-packages\contextily\tile.py:614: UserWarning: The inferred zoom level of 20 is not valid for the current tile provider (valid zooms: 0 - 19).
  warnings.warn(msg)



Processing: 752 / 763: ze6gfwt2C_4q1yTlUeCH5w_0.0_0.0_0_140.76_L135.jpg
skip the backward thumbnail:  L135 ze6gfwt2C_4q1yTlUeCH5w_0.0_0.0_0_140.76_L135.pkl.bz2
    No image_pairs, skip. 

Processing: 753 / 763: ze6gfwt2C_4q1yTlUeCH5w_0.0_0.0_0_185.76_L90.jpg
    image pairs: ('ze6gfwt2C_4q1yTlUeCH5w_0.0_0.0_0_185.76_L90.pkl.bz2', 'k5-O4GqXpRuptfifbbEKiA_0.0_0.0_0_318.49_L135.pkl.bz2')

pair[1]: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\street_image_mapping\Heyward_st_pano_thumnail_segmented\k5-O4GqXpRuptfifbbEKiA_0.0_0.0_0_318.49_L135.pkl.bz2
    No trunk pairs.
Processing: 754 / 763: ze6gfwt2C_4q1yTlUeCH5w_0.0_0.0_0_230.76_L45.jpg
    image pairs: ('ze6gfwt2C_4q1yTlUeCH5w_0.0_0.0_0_230.76_L45.pkl.bz2', 'k5-O4GqXpRuptfifbbEKiA_0.0_0.0_0_318.49_L135.pkl.bz2')
    image pairs: ('ze6gfwt2C_4q1yTlUeCH5w_0.0_0.0_0_230.76_L45.pkl.bz2', 'k5-O4GqXpRuptfifbbEKiA_0.0_0.0_0_3.49_L90.pkl.bz2')

pair[1]: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\R

C:\Users\USER\AppData\Local\Temp\ipykernel_20996\198553039.py:113: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  localized_trunk_gdf.to_file(localized_trunk_fname)


In [25]:
trunk_pairs_df

In [29]:
all_trunk_gdf

,index,distance,root_center_col,root_bottom_row,azimuth_rad,altitude_rad,depth_h_distance,median_wid,median_wid_row,median_row_start_col,median_row_end_col,panoID,depth_x,depth_y,pano_x,pano_y,pano_heading_deg,thumbnail_heading_deg,trunk_heading_deg,image_file,azimuth_deg,geometry,Type
0,0,16.328520,222,447,-0.515352,-0.106659,16.235730,33.0,417,207,239,vFu4cCea3kkheGt941HrBA,609330.682129,239253.802120,609325.280561,239238.491274,93.959946,48.96,19.432513,vFu4cCea3kkheGt941HrBA_0.0_0.0_0_48.96_L45.jpg,-29.527487,POINT (609330.682 239253.802),trunk
1,0,11.061842,878,519,0.620619,-0.211300,10.815816,32.0,453,860,891,Zz5dleaUI9Wt2KJ1IxhLxg,609341.742968,239246.498196,609334.889684,239238.130736,93.763100,3.76,39.318844,Zz5dleaUI9Wt2KJ1IxhLxg_0.0_0.0_0_3.76_L90.jpg,35.558844,POINT (609341.743 239246.498),trunk


In [12]:
trunk_pairs_df

,index,distance,root_center_col,root_bottom_row,azimuth_rad,altitude_rad,depth_h_distance,median_wid,median_wid_row,median_row_start_col,median_row_end_col,panoID,depth_x,depth_y,pano_x,pano_y,pano_heading_deg,thumbnail_heading_deg,trunk_heading_deg,image_file,azimuth_deg,geometry,Type
0,0,10.249231,38,538,-0.746878,-0.217234,10.008346,45.0,467,14,58,L9Jp_XEhrMmgm35YhruiRQ,608658.094720,239254.141746,608665.849204,239260.469065,183.581512,273.58,230.787061,L9Jp_XEhrMmgm35YhruiRQ_0.0_0.0_0_273.58_R90.jpg,-42.792939,POINT (608658.095 239254.142),trunk
1,1,9.870755,870,559,0.610201,-0.273114,9.504901,22.0,473,859,880,L9Jp_XEhrMmgm35YhruiRQ,608658.414927,239266.391452,608665.849204,239260.469065,183.581512,273.58,308.541967,L9Jp_XEhrMmgm35YhruiRQ_0.0_0.0_0_273.58_R90.jpg,34.961967,POINT (608658.415 239266.391),trunk


In [ ]:
localized_trunk_gdf.explore()

In [ ]:
geometry = gpd.points_from_xy(all_pair_trunk_df['tri_x'], all_pair_trunk_df['tri_y'])
localized_trunk_gdf = gpd.GeoDataFrame(data=all_pair_trunk_df, geometry=geometry).set_crs(local_crs)
localized_trunk_fname = os.path.join(save_dir, "localized_trunks.shp")
localized_trunk_gdf.to_file(localized_trunk_fname)

In [ ]:
def show_image_pair(image_pair, image_dir, camera_dis_thres=20):
    images = []
    basenames = []
    for detect_file in image_pair:
        basename = os.path.basename(detect_file).replace('.pkl.bz2', '.jpg')
        basenames.append(basename[:-4])
        image_fname = os.path.join(image_dir, basename)
        image = cv2.imread(image_fname)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        images.append(image_rgb)

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 14))

    for idx, image in enumerate(images):
        try:
            # df = trunk_pair_dfs[idx].query(f" depth_h_distance < {camera_dis_thres} ")
            axs[idx].imshow(image)
            # axs[idx].plot(df['root_center_col'], df['root_bottom_row'], 'o', color='red')
            axs[idx].set_title(basenames[idx])
        except:
            pass

# camera_dis_thres = 30
# show_image_pair(image_pair=image_pairs[0], image_dir=image_dir, camera_dis_thres=camera_dis_thres)

In [ ]:
gdf_list = []

for df in [trunk_df, paired_trunk_df]:
    # for trunk initial location from depthmap
    geometry = gpd.points_from_xy(df['depth_x'], df['depth_y'])
    gdf = gpd.GeoDataFrame(data=df, geometry=geometry).set_crs(local_crs)
    gdf['Type'] = 'trunk'
    gdf_list.append(gdf)

    # for pano (camera) location
    geometry = gpd.points_from_xy(df['pano_x'], df['pano_y'])
    gdf = gpd.GeoDataFrame(data=df.iloc[:1], geometry=geometry[:1]).set_crs(local_crs)
    gdf['Type'] = 'camera'
    gdf_list.append(gdf)


# all_gdf = pd.concat(gdf_list)

# all_gdf.explore(column='Type', cmap='seismic', marker_kwds={'radius':5})  # column='type', cmap='seismic',

In [ ]:
# all_gdf.query("type == 'trunk' ").sort_values('x')

near_trunk_thres = 3



all_trunk_gdf = all_gdf.query("Type == 'trunk' ")
all_camera_gdf = all_gdf.query("Type == 'camera' ")

camera_dis_thres = 20

# trunk_pair_df = tri.form_trunk_pairs(all_trunk_gdf, near_thres=3, camera_dis_thres=camera_dis_thres)
# trunk_pair_df

# Show all images

In [ ]:
def show_paried_trunk(trunk_pair_df, image_dir):
    images = []
    basenames = []
    image_pair = trunk_pair_df.iloc[0:2]['image_file'].to_list()

    for image_file in image_pair:
        basename = os.path.basename(image_file)
        basenames.append(basename)
        image_fname = os.path.join(image_dir, image_file)
        image = cv2.imread(image_fname)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        images.append(image_rgb)


    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 14))

    for idx, image in enumerate(images):

        try:
            df = trunk_pair_df.iloc[idx:idx + 1]
            axs[idx].imshow(image)
            axs[idx].plot(df['root_center_col'], df['root_bottom_row'], 'o', color='red')
            axs[idx].plot(df['root_center_col'], df['median_wid_row'], 'o', color='y')
            axs[idx].set_title(basenames[idx])

        except Exception as e:
            print("Error:", e)
            continue

# show_paried_trunk(trunk_pair_df=trunk_pair_df.iloc[:], image_dir=image_dir)

In [ ]:
trunk_pairs_df

In [ ]:
import matplotlib.pyplot as plt





In [ ]:
a_pair_trunk_df

In [ ]:
trunk_pair_df
all_camera_gdf

In [ ]:
# print(pano.panoID, "heading_dif_deg:", heading_dif_deg)


fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))

ax.set_ylim(trunk_pair_df['depth_y'].min() - 20, trunk_pair_df['depth_y'].max() + 20)
ax.set_xlim(trunk_pair_df['depth_x'].min() - 20, trunk_pair_df['depth_x'].max() + 20)

for idx, pano in trunk_pair_df.iloc[:2].iterrows():
    tri.draw_yaw_arrow(ax, pano, length=3, width=0.3)
    # break

In [ ]:


trunk_pair_df = tri.griangulate_trunk_location(trunk_pair_df.iloc[:2].copy())
trunk_pair_df


In [ ]:


trunk_pair_df = compute_diameter(trunk_pair_df.iloc[:2].copy())
trunk_pair_df[['h_distance_to_camera', 'median_wid', 'diameter']]


In [ ]:
# pairs_df = form_trunk_pairs(all_trunk_gdf, near_thres=3, camera_dis_thres=15)

In [ ]:
def show_image_pair(image_pair, trunk_pair_dfs, image_dir, camera_dis_thres=20):
    images = []
    basenames = []
    for detect_file in image_pair:
        basename = os.path.basename(detect_file).replace('.pkl.bz2', '.jpg')
        basenames.append(basename[:-4])
        image_fname = os.path.join(image_dir, basename)
        image = cv2.imread(image_fname)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        images.append(image_rgb)

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 14))

    for idx, image in enumerate(images):
        try:
            df = trunk_pair_dfs[idx].query(f" depth_h_distance < {camera_dis_thres} ")
            axs[idx].imshow(image)
            axs[idx].plot(df['root_center_col'], df['root_bottom_row'], 'o', color='red')
            axs[idx].set_title(basenames[idx])
        except:
            pass

show_image_pair(image_pair=image_pairs[1], trunk_pair_dfs=[trunk_df, paired_trunk_df], image_dir=image_dir, camera_dis_thres=camera_dis_thres)

In [ ]:
image = cv2.imread(image_fname)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
annotated_image = tri.show_detection_segment(image=image, detections=detection[1])

In [ ]:

pano_meta


In [ ]:
# pano_obj.calculate_xy(pano_meta)

In [ ]:
# Try to use the bottom ten row of depthmap as the distance
root_ground_height = 10 # unit: row

bottom_row = idx_row.max()
top_row = max(idx_row.max() - root_ground_height, 0)
bottom_row, top_row

In [ ]:
len(idx_row)

In [ ]:
root_row_idx = np.where(np.logical_and(top_row < idx_row, idx_row < bottom_row))
root_row_idx

In [ ]:
root_idx

In [ ]:
# the distance of from camera to trunk bottom.
root_idx = row_col_idx[root_row_idx]
depthmap[root_idx[:, 0], root_idx[:, 1]].mean()

In [ ]:
depthmap[root_idx[:, 0], root_idx[:, 1]].max(), depthmap[root_idx[:, 0], root_idx[:, 1]].min()

In [ ]:
(trunk_mask * depthmap)

In [ ]:
plt.hist(depthmap[trunk_mask])
depthmap[trunk_mask].mean()
indices = np.where(trunk_mask > 0)
# print("Indices:", list(zip(indices[0], indices[1])))
# list(zip(indices[0], indices[1]))
# indices[0][0]

In [ ]:
mask = np.zeros_like(depthmap)
highlighted_depthmap = depthmap.copy()
highlighted_depthmap[root_idx[:, 0], root_idx[:, 1]] = 50

plt.imshow(highlighted_depthmap)

In [ ]:
# depthmap = cv2.imread(depthmap_fname)[:, :, 0]
# root_idx
# mask = depthmap[root_idx[:, 0], root_idx[:, 1]]
plt.imshow(depthmap)

In [ ]:
# image = Image.open(depthmap_fname)
# image = Image.open(image_fname)
image = cv2.imread(image_fname)
tri.show_detection_segment(image=image, detections=detection[1])

In [ ]:
plt.imshow(image)

In [ ]:
plt.imshow(masks[0])


In [ ]:
detection[1]